In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
date_parser = lambda c: pd.to_datetime(c, format='%d/%m/%Y %H:%M:%S', errors='coerce')
data = pd.read_csv('ITSM_data.csv', parse_dates=['Open_Time','Reopen_Time','Close_Time','Resolved_Time'],)

/Users/ashok/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head(2)

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,Unnamed: 9,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.820967,...,NaT,2013-04-11 13:50:27,2013-04-11 13:51:17,"3871,691111",Other,1.0,SD0000007,2.0,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.936566,...,2013-02-12 12:31:02,2013-02-12 12:36:21,2013-02-12 12:36:25,"4354,786389",Software,1.0,SD0000011,1.0,NaN,NaN


In [4]:
data.Close_Time[3]

Timestamp('2013-11-14 09:31:24')

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46606 entries, 0 to 46605
Data columns (total 27 columns):
CI_Name                       46606 non-null object
CI_Cat                        46495 non-null object
CI_Subcat                     46495 non-null object
WBS                           46606 non-null object
Incident_ID                   46606 non-null object
Status                        46606 non-null object
Impact                        46606 non-null object
Urgency                       46606 non-null object
Priority                      45226 non-null float64
Unnamed: 9                    46606 non-null float64
Category                      46606 non-null object
KB_number                     46606 non-null object
Alert_Status                  46606 non-null object
No_of_Reassignments           46605 non-null float64
Unnamed: 14                   46606 non-null int64
Open_Time                     46606 non-null datetime64[ns]
Reopen_Time                   2284 non-null datet

In [6]:
Counter(data.Category).most_common(5)

[('incident', 37748),
 ('request for information', 8846),
 ('complaint', 11),
 ('request for change', 1)]

In [7]:
data.loc[30:35,['Open_Time','Reopen_Time','Close_Time','Resolved_Time','Handle_Time_hrs','Priority']]

,Open_Time,Reopen_Time,Close_Time,Resolved_Time,Handle_Time_hrs,Priority
30,2013-02-19 11:52:03,NaT,2013-12-23 13:04:33,2013-12-23 13:03:51,"2050,36",4.0
31,2013-02-20 10:14:37,NaT,2013-08-11 13:57:15,2013-08-11 13:56:54,"1935,710556",4.0
32,2013-02-25 16:03:52,NaT,2014-12-03 10:16:22,2014-12-03 10:15:40,"2764,633056",4.0
33,2013-02-26 16:44:41,NaT,2013-12-11 13:40:13,2013-12-11 13:40:07,"6,1125",4.0
34,2013-02-28 12:23:11,2013-03-12 08:57:55,2013-03-12 10:36:32,2013-03-12 10:36:18,"88,67166667",4.0
35,2013-02-28 16:49:58,NaT,2013-11-20 16:38:20,2013-11-20 16:38:17,"1917,175278",4.0


In [8]:
data[data.Priority.isnull()].Incident_ID.count()

1380

In [9]:
data[data.Impact.isnull()].Incident_ID.count()

0

In [10]:
data1 = data[data.Priority.isnull()==False]

In [11]:
data1.shape

(45226, 26)

In [12]:
#SMOTE = Synthetic Minority Oversampling Technique

In [13]:
Counter(data1.Priority)

Counter({1.0: 3, 2.0: 697, 3.0: 5323, 4.0: 22717, 5.0: 16486})

In [14]:
#find Missing Priorities.  For you we need to find Missing Impact then using Impact and Urgency, derive Priority using Matrix

In [15]:
import datetime as dt

In [16]:
duration = []
for index,row in data.iterrows():
    duration.append(row.Close_Time-row.Open_Time)


In [17]:
data['duration'] = duration

In [18]:
data.duration.describe()

count                      46606
mean      6 days 06:29:16.584795
std      81 days 16:12:46.045016
min          -334 days +17:48:02
25%       0 days 00:40:08.250000
50%              0 days 04:32:14
75%       6 days 22:42:52.250000
max            781 days 21:26:08
Name: duration, dtype: object

In [19]:
%matplotlib inline
data[data['duration']> dt.timedelta(days=30)].duration.describe()

count                        8298
mean     107 days 11:29:38.274765
std       84 days 22:10:51.373665
min              30 days 00:02:00
25%              32 days 11:57:22
50%              89 days 04:08:03
75%             153 days 00:33:00
max             781 days 21:26:08
Name: duration, dtype: object

In [20]:
data[data['duration']< dt.timedelta(days=30)].duration.describe()

count                        38308
mean     -16 days +08:20:03.635454
std        62 days 02:00:04.795431
min            -334 days +17:48:02
25%                0 days 00:26:47
50%         0 days 02:15:52.500000
75%         1 days 00:45:43.750000
max               29 days 23:59:55
Name: duration, dtype: object

#### Ticket statuses
It's broken (Open, reopened and in-progress) <br>
I think I've fixed it (Resolved)<br>
Yes, it's fixed, never look at it again (Closed)

In [21]:
data[data.Close_Time<data.Open_Time].loc[:,['Close_Time','Open_Time','Resolved_Time']]

,Close_Time,Open_Time,Resolved_Time
39,2013-09-10 13:41:13,2013-11-03 09:08:37,2013-09-10 13:41:08
40,2013-08-11 15:34:40,2013-12-03 09:54:11,2013-08-11 15:34:37
42,2013-01-10 07:52:46,2013-03-21 14:54:33,2013-01-10 07:52:43
49,2013-08-11 15:36:34,2013-09-04 12:49:06,2013-08-11 15:36:30
50,2013-08-11 15:41:25,2013-10-04 13:20:16,2013-08-11 15:41:22
51,2013-08-11 15:46:30,2013-10-04 15:13:52,2013-08-11 15:46:27
70,2013-05-12 11:19:20,2013-07-05 14:04:27,2013-05-12 11:19:14
72,2013-05-12 11:16:27,2013-07-05 15:12:56,2013-05-12 11:15:55
84,2013-05-12 08:28:36,2013-05-24 09:34:42,2013-05-12 08:27:23
87,2013-05-11 10:36:50,2013-05-27 16:28:37,2013-05-11 10:36:48


In [22]:
#Close Time can't be before Open Time. This takes care of Invalid close times
datav = data[data.Close_Time>data.Open_Time]

In [23]:
datav.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42709 entries, 0 to 46605
Data columns (total 27 columns):
CI_Name                       42709 non-null object
CI_Cat                        42603 non-null object
CI_Subcat                     42603 non-null object
WBS                           42709 non-null object
Incident_ID                   42709 non-null object
Status                        42709 non-null object
Impact                        42709 non-null object
Urgency                       42709 non-null object
Priority                      41406 non-null float64
Unnamed: 9                    42709 non-null float64
Category                      42709 non-null object
KB_number                     42709 non-null object
Alert_Status                  42709 non-null object
No_of_Reassignments           42708 non-null float64
Unnamed: 14                   42709 non-null int64
Open_Time                     42709 non-null datetime64[ns]
Reopen_Time                   1875 non-null datet

In [24]:
datav[datav['duration']> dt.timedelta(days=30)].duration.describe()

count                        8298
mean     107 days 11:29:38.274765
std       84 days 22:10:51.373665
min              30 days 00:02:00
25%              32 days 11:57:22
50%              89 days 04:08:03
75%             153 days 00:33:00
max             781 days 21:26:08
Name: duration, dtype: object

In [25]:
Counter(datav.CI_Cat).most_common(10)

[('application', 30094),
 ('subapplication', 7338),
 ('computer', 3235),
 ('storage', 652),
 ('hardware', 340),
 ('software', 311),
 ('database', 205),
 ('displaydevice', 180),
 ('officeelectronics', 147),
 (nan, 106)]

In [26]:
Counter(datav.CI_Subcat).most_common(10)

[('Server Based Application', 17365),
 ('Web Based Application', 14159),
 ('Desktop Application', 3481),
 ('Laptop', 1632),
 ('SAP', 1053),
 ('Banking Device', 877),
 ('Citrix', 730),
 ('SAN', 501),
 ('Client Based Application', 435),
 ('Desktop', 365)]

In [27]:
Counter(datav.WBS).most_common(10)

[('WBS000073', 12497),
 ('WBS000091', 2151),
 ('WBS000263', 2031),
 ('WBS000072', 1970),
 ('WBS000095', 1078),
 ('WBS000271', 1041),
 ('WBS000152', 995),
 ('WBS000318', 979),
 ('WBS000146', 828),
 ('WBS000094', 797)]

In [28]:
Counter(datav.Category).most_common(5)

[('incident', 34715),
 ('request for information', 7984),
 ('complaint', 9),
 ('request for change', 1)]

In [29]:
selected = data.loc[:,['CI_Subcat','WBS','Category','Priority']]
selected.dropna(inplace=True)
Counter(selected.Priority)

Counter({1.0: 3, 2.0: 689, 3.0: 5310, 4.0: 22689, 5.0: 16427})

In [30]:
selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45118 entries, 0 to 46605
Data columns (total 4 columns):
CI_Subcat    45118 non-null object
WBS          45118 non-null object
Category     45118 non-null object
Priority     45118 non-null float64
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [31]:
selected = selected[selected.Priority!=1]

In [32]:
selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45115 entries, 0 to 46605
Data columns (total 4 columns):
CI_Subcat    45115 non-null object
WBS          45115 non-null object
Category     45115 non-null object
Priority     45115 non-null float64
dtypes: float64(1), object(3)
memory usage: 1.7+ MB


In [88]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
selected.CI_Subcat = enc.fit_transform(X.CI_Subcat)
selected.Category = enc.fit_transform(X.Category)
selected.WBS = selected.WBS.str[4:].astype('int')

In [89]:
Counter(selected.Category)

Counter({0: 11, 1: 36316, 2: 1, 3: 8787})

In [90]:
X = selected.loc[:,['CI_Subcat','WBS','Category']]
y = selected.Priority


In [37]:
pd.DataFrame(enc.classes_)

,0
0,complaint
1,incident
2,request for change
3,request for information


In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,random_state=13)


In [39]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

/Users/ashok/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [40]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[ 101   13   43   15]
 [  53  811  363  101]
 [  54  173 5242  203]
 [   5   46  779 3277]]


0.8361556875609539

In [41]:
model=RandomForestClassifier(max_depth=15)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[ 103   12   42   15]
 [  54  799  371  104]
 [  54  194 5177  247]
 [   5   43  738 3321]]


0.8334072169518575

In [42]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[  99   10   52   11]
 [  52  614  511  151]
 [  53  163 5181  275]
 [   5   28  875 3199]]


/Users/ashok/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8061884918875787

In [ ]:
grid_model.best_params_

In [ ]:
from sklearn.svm import SVC

model=SVC(kernel='rbf',C=10,gamma=0.1)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
pd.crosstab(y_test,y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = { 'kernel':['rbf'],
                'C':[1,100],
                'gamma' : [0.01,1.0]}
grid_model = GridSearchCV(SVC(),parameters)
grid_model.fit(X_train, y_train)
grid_model.best_score_

In [55]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,accuracy_score
print(classification_report(y_test,y_pred))


             precision    recall  f1-score   support

        2.0       0.47      0.58      0.52       172
        3.0       0.75      0.46      0.57      1328
        4.0       0.78      0.91      0.84      5672
        5.0       0.88      0.78      0.83      4107

avg / total       0.81      0.81      0.80     11279



In [56]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_sample(X_train,y_train)

In [75]:
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test.values)
print(accuracy_score(y_test,y_pred))
pd.crosstab(y_test,y_pred)

0.8372196116677011


col_0,2.0,3.0,4.0,5.0
Priority,,,,
2.0,103,11,42,16
3.0,54,812,358,104
4.0,54,172,5186,260
5.0,5,48,712,3342


In [76]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        2.0       0.48      0.60      0.53       172
        3.0       0.78      0.61      0.68      1328
        4.0       0.82      0.91      0.87      5672
        5.0       0.90      0.81      0.85      4107

avg / total       0.84      0.84      0.84     11279



In [84]:
Counter(y)

Counter({2.0: 689, 3.0: 5310, 4.0: 22689, 5.0: 16427})

In [87]:
selected.head()

,CI_Subcat,WBS,Category,Priority
0,Web Based Application,WBS000162,incident,4.0
1,Web Based Application,WBS000088,incident,3.0
3,Web Based Application,WBS000088,incident,4.0
4,Web Based Application,WBS000088,incident,4.0
5,Web Based Application,WBS000088,incident,4.0


In [94]:
from sklearn.externals import joblib
joblib.dump(model,'ML_perdict_high_priority.ml')

['ML_perdict_high_priority.ml']